# Import

In [2]:
import pandas as pd

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

ModuleNotFoundError: Module langchain_community.embeddings not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

# Data Load & Pre-processing

In [3]:
train = pd.read_csv('./data/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('./data/test.csv', encoding = 'utf-8-sig')

In [4]:
# 데이터 전처리
train['공사종류(대분류)'] = train['공사종류'].str.split(' / ').str[0]
train['공사종류(중분류)'] = train['공사종류'].str.split(' / ').str[1]
train['공종(대분류)'] = train['공종'].str.split(' > ').str[0]
train['공종(중분류)'] = train['공종'].str.split(' > ').str[1]
train['사고객체(대분류)'] = train['사고객체'].str.split(' > ').str[0]
train['사고객체(중분류)'] = train['사고객체'].str.split(' > ').str[1]

test['공사종류(대분류)'] = test['공사종류'].str.split(' / ').str[0]
test['공사종류(중분류)'] = test['공사종류'].str.split(' / ').str[1]
test['공종(대분류)'] = test['공종'].str.split(' > ').str[0]
test['공종(중분류)'] = test['공종'].str.split(' > ').str[1]
test['사고객체(대분류)'] = test['사고객체'].str.split(' > ').str[0]
test['사고객체(중분류)'] = test['사고객체'].str.split(' > ').str[1]

In [5]:
# 훈련 데이터 통합 생성
combined_training_data = train.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        ),
        "answer": row["재발방지대책 및 향후조치계획"]
    },
    axis=1
)

# DataFrame으로 변환
combined_training_data = pd.DataFrame(list(combined_training_data))

In [6]:
# 테스트 데이터 통합 생성
combined_test_data = test.apply(
    lambda row: {
        "question": (
            f"공사종류 대분류 '{row['공사종류(대분류)']}', 중분류 '{row['공사종류(중분류)']}' 공사 중 "
            f"공종 대분류 '{row['공종(대분류)']}', 중분류 '{row['공종(중분류)']}' 작업에서 "
            f"사고객체 '{row['사고객체(대분류)']}'(중분류: '{row['사고객체(중분류)']}')와 관련된 사고가 발생했습니다. "
            f"작업 프로세스는 '{row['작업프로세스']}'이며, 사고 원인은 '{row['사고원인']}'입니다. "
            f"재발 방지 대책 및 향후 조치 계획은 무엇인가요?"
        )
    },
    axis=1
)

# DataFrame으로 변환
combined_test_data = pd.DataFrame(list(combined_test_data))

# Model import

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [2]:
model_id = "NCSOFT/Llama-VARCO-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

NameError: name 'AutoTokenizer' is not defined

# Vector store 생성

In [10]:
# Train 데이터 준비
train_questions_prevention = combined_training_data['question'].tolist()
train_answers_prevention = combined_training_data['answer'].tolist()

train_documents = [
    f"Q: {q1}\nA: {a1}" 
    for q1, a1 in zip(train_questions_prevention, train_answers_prevention)
]

# 임베딩 생성
embedding_model_name = "jhgan/ko-sbert-nli"  # 임베딩 모델 선택
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)

# 벡터 스토어에 문서 추가
vector_store = FAISS.from_texts(train_documents, embedding)

# Retriever 정의
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})


# RAG chain 생성

In [12]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,  # sampling 활성화
    temperature=0.1,
    return_full_text=False,
    max_new_tokens=64,
)

prompt_template = """
### 지침: 당신은 건설 안전 전문가입니다.
질문에 대한 답변을 핵심 내용만 요약하여 간략하게 작성하세요.
- 서론, 배경 설명 또는 추가 설명을 절대 포함하지 마세요.
- 다음과 같은 조치를 취할 것을 제안합니다: 와 같은 내용을 포함하지 마세요.

{context}

### 질문:
{question}

[/INST]

"""

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# 커스텀 프롬프트 생성
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


# RAG 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  
    chain_type="stuff",  # 단순 컨텍스트 결합 방식 사용
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}  # 커스텀 프롬프트 적용
)

Device set to use cuda:0


# Inference

In [15]:
# 테스트 실행 및 결과 저장
test_results = []

print("테스트 실행 시작... 총 테스트 샘플 수:", len(combined_test_data))

for idx, row in combined_test_data.iterrows():
    # # 50개당 한 번 진행 상황 출력
    # if (idx + 1) % 50 == 0 or idx == 0:
    #     print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    print(f"\n[샘플 {idx + 1}/{len(combined_test_data)}] 진행 중...")
    
    # RAG 체인 호출 및 결과 생성
    prevention_result = qa_chain.invoke(row['question'])

    # 결과 저장
    result_text = prevention_result['result']
    test_results.append(result_text)
    print(row['question'])
    print(result_text, "\n")
    
    # 10개만 출력
    if idx == 9:
        break

print("\n테스트 실행 완료! 총 결과 수:", len(test_results))

테스트 실행 시작... 총 테스트 샘플 수: 964

[샘플 1/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '철근콘크리트공사' 작업에서 사고객체 '건설기계'(중분류: '콘크리트펌프')와 관련된 사고가 발생했습니다. 작업 프로세스는 '타설작업'이며, 사고 원인은 '펌프카 아웃트리거 바닥 고임목을 3단으로 보강 했음에도, 지반 침하(아웃트리거 우측 상부 1개소)가 발생하였고,  좌, 우측 아웃트리거의 펼친 길이가 상이하고 타설 위치가 건물 끝부분 모서리에 위치하여 붐대호스를 최대로 펼치다 보니 장비에 대한 무게중심이 한쪽으로 쏠려 일부 전도되는 사고가 발생된 것으로 판단됨'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
### 답변:
사고 원인 분석 결과, 아웃트리거 보강의 부족과 지반 침하, 아웃트리거의 불균형한 펼침, 무게중심의 불균형 등이 사고 원인으로 작용했다고 판단됩니다.

재 


[샘플 2/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '수장공사' 작업에서 사고객체 '건설공구'(중분류: '공구류')와 관련된 사고가 발생했습니다. 작업 프로세스는 '절단작업'이며, 사고 원인은 '작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
### 답변:
1. 절단기 사용 시 안전 교육 강화: 작업자들에게 절단기 안전사용법과 안전장비 착용의 중요성에 대해 철저히 교육해야 합니다.
2. 안전장비 착용 강제화: 절단 작업 시 반드시 보안면 


[샘플 3/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '미장공사' 작업에서 사고객체 '기타'(중분류: '기타')와 관련된 사고가 발생했습니다. 작업 프로세스는 '이동'이며, 사고 원인은 '작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '조적공사' 작업에서 사고객체 '건설자재'(중분류: '자재')와 관련된 사고가 발생했습니다. 작업 프로세스는 '기타'이며, 사고 원인은 '작업 발판 위 벽돌 잔재를 밟고 넘어짐'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
### 답변:
주변 정리 철저, 안전 관리 강화, 작업자 안전 의식 고취를 위한 지속적인 지도 요청이 필요합니다.

사고 원인인 '작업 발판 위 벽돌 잔재를 밟고 넘어짐'은 작업장 내 정리정 


[샘플 5/964] 진행 중...
공사종류 대분류 '토목', 중분류 '교량' 공사 중 공종 대분류 '토목', 중분류 '교량공사' 작업에서 사고객체 '기타'(중분류: '기타')와 관련된 사고가 발생했습니다. 작업 프로세스는 '해체작업'이며, 사고 원인은 '점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
### 답변:
1. 작업자에게 점심 휴식 시간에 안전한 이동 경로 및 장소를 교육하고, 사고 예방을 위한 주의사항을 강조하는 안전 교육 실시.
2. 작업장 내 안전 시설물(예: 안전 램프, 비상구 등)을 


[샘플 6/964] 진행 중...
공사종류 대분류 '건축', 중분류 '건축물' 공사 중 공종 대분류 '건축', 중분류 '가설공사' 작업에서 사고객체 '가시설'(중분류: '거푸집')와 관련된 사고가 발생했습니다. 작업 프로세스는 '설치작업'이며, 사고 원인은 '안전발판 설치 미흡 및 작업절차 미준수'입니다. 재발 방지 대책 및 향후 조치 계획은 무엇인가요?
### 답변:
안전발판 전 수리작업 실시, 작업절차 개정 및 교육 실시, 정기적 안전검사 강화, 작업자별 안전성적평가 실시, 안전관리체계 개선 및 정기적 검토.

위 사고의 원인인 '안 


[샘플 7/964] 진행 중...
공사종류 대분류 '토목', 중분류 '도로' 공사 중 공종 대분류 '토목', 중분류 '관공사 부대공사' 작업에

# Submission

In [16]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(test_results)
print(pred_embeddings.shape)  # (샘플 개수, 768)

c:\Users\user\miniconda3\envs\deeplearning\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--jhgan--ko-sbert-sts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


(10, 768)


In [19]:
test_results

['### 답변:\n사고 원인 분석 결과, 아웃트리거 보강의 부족과 지반 침하, 아웃트리거의 불균형한 펼침, 무게중심의 불균형 등이 사고 원인으로 작용했다고 판단됩니다.\n\n재',
 '### 답변:\n1. 절단기 사용 시 안전 교육 강화: 작업자들에게 절단기 안전사용법과 안전장비 착용의 중요성에 대해 철저히 교육해야 합니다.\n2. 안전장비 착용 강제화: 절단 작업 시 반드시 보안면',
 '### 답변:\n1. 미장공사 작업 중 주의가 필요한 계단 이동 경로를 명확히 표시하고, 근로자들에게 전방 주시의 중요성을 강조하는 안전 교육을 실시합니다.\n2. 계단 사용 시 안전모를 착용하도록 권고하고,',
 "### 답변:\n주변 정리 철저, 안전 관리 강화, 작업자 안전 의식 고취를 위한 지속적인 지도 요청이 필요합니다.\n\n사고 원인인 '작업 발판 위 벽돌 잔재를 밟고 넘어짐'은 작업장 내 정리정",
 '### 답변:\n1. 작업자에게 점심 휴식 시간에 안전한 이동 경로 및 장소를 교육하고, 사고 예방을 위한 주의사항을 강조하는 안전 교육 실시.\n2. 작업장 내 안전 시설물(예: 안전 램프, 비상구 등)을',
 "### 답변:\n안전발판 전 수리작업 실시, 작업절차 개정 및 교육 실시, 정기적 안전검사 강화, 작업자별 안전성적평가 실시, 안전관리체계 개선 및 정기적 검토.\n\n위 사고의 원인인 '안",
 '### 답변:\n1. 작업자 교육 강화: 작업자의 안전 의식을 높이기 위해 정기적인 안전 교육을 실시하고, 사고 원인에 대한 심층 이해를 도모해야 합니다.\n2. 작업 환경 개선: 작업장 내 안전 표지판 설치, 작업 공간 정리',
 '### 답변:\n1. TSC GIRDER 조립 시 SPLICE PLATE 설치 작업의 위험성 평가 실시.\n2. 작업장비 사용자 교육 강화: 안전장비 착용, 적절한 작업 자세, 안전 거리 유지 등.\n3. 작업장 내 안전시설 개선',
 '### 답변:\n\n안전사고 재발 방지를 위해 다음과 같은 대책을 제안합니다:\n\n1.

In [20]:
submission = pd.read_csv('./sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
# submission.iloc[:,1] = test_results
# submission.iloc[:,2:] = pred_embeddings
submission.iloc[:10,1] = test_results
submission.iloc[:10,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('./baseline_submission.csv', index=False, encoding='utf-8-sig')